In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import json

transactions = []
days_in_month = 31
for i in range (1,days_in_month+1):
  for line in open('data/return-data/2017-01-'+str(i).zfill(2)+'.txt', 'r'):
    transactions.append(json.loads(line))
transactions_df = pd.DataFrame.from_dict(transactions)
transactions_df.sample(5)

,basket,returnLabel,totalAmount,transactionId,zipCode
13599,"[3, 2, 3, 2, 5, 5, 5, 3, 0, 3]",0,310,8278197193,2479
5768,"[3, 2, 0, 1, 4, 5, 4, 5, 3]",0,342,4295911772,4290
30475,"[1, 3, 0]",0,102,6437277708,2904
289,"[0, 1, 1, 2, 3]",0,105,6888155391,1279
29258,"[0, 4, 3]",0,141,6260576268,4117


## Use the pandas method "pd.get_dummies" to apply one-hot encoding on the "zip-code" column.

In [2]:
one_hot_df = pd.get_dummies(transactions_df.zipCode)

## Use the newly generated columns as new features in your ML model and re-train it (you have to drop the original "zipCode" column before that).

In [3]:
transactions_df = transactions_df.drop('zipCode', axis=1)
transactions_df_one_hot = pd.concat([transactions_df, one_hot_df], axis=1)
transactions_df_one_hot.sample(5)

,basket,returnLabel,totalAmount,transactionId,1000,1001,1002,1003,1004,1005,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
15055,"[0, 3, 2, 4, 3]",0,115,8272956460,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6624,"[3, 2, 2, 2, 0, 4, 5]",0,343,5601312856,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9299,"[4, 2, 5, 4, 2, 4]",0,738,3950645650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17333,"[2, 5, 0, 5, 2, 0]",0,330,6825854337,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20762,"[0, 3, 0]",0,30,1160776063,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split

X = one_hot_df
y = transactions_df_one_hot.returnLabel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
one_hot_df.sample(5)

,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
13486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(solver='lbfgs')
logmodel.fit(X_train, y_train)

predictions = logmodel.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

print(classification_report(y_test,predictions))
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      9024
           1       0.00      0.00      0.00       276

   micro avg       0.97      0.97      0.97      9300
   macro avg       0.49      0.50      0.49      9300
weighted avg       0.94      0.97      0.96      9300

Accuracy: 0.9703225806451613


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Third Version: totalAmount + newly generated columns

In [7]:
third_version_df = pd.concat([transactions_df.totalAmount, one_hot_df], axis=1)

X = third_version_df
y = transactions_df_one_hot.returnLabel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
third_version_df.sample(5)

,totalAmount,1000,1001,1002,1003,1004,1005,1006,1007,1008,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
15886,540,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20325,492,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23518,315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9505,106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2057,50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(solver='lbfgs')
logmodel.fit(X_train, y_train)

predictions = logmodel.predict(X_test)

In [9]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

print(classification_report(y_test,predictions))
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      9004
           1       0.00      0.00      0.00       296

   micro avg       0.97      0.97      0.97      9300
   macro avg       0.48      0.50      0.49      9300
weighted avg       0.94      0.97      0.95      9300

Accuracy: 0.9681720430107527


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
